In [1]:
import os
import time

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.preprocessing import image
from glob import glob
from tqdm import tqdm
from sklearn.utils import shuffle
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras.models import Sequential, Model
from keras.layers import BatchNormalization
from keras import regularizers, applications, optimizers, initializers
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from tensorflow.keras.applications.densenet import DenseNet201
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping

In [2]:
def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))

def precision_threshold(threshold = 0.5):
    def precision(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(y_pred)
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision

def recall_threshold(threshold = 0.5):
    def recall(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.clip(y_true, 0, 1))
        recall_ratio = true_positives / (possible_positives + K.epsilon())
        return recall_ratio
    return recall

def fbeta_score_threshold(beta = 1, threshold = 0.5):
    def fbeta_score(y_true, y_pred):
        threshold_value = threshold
        beta_value = beta
        p = precision_threshold(threshold_value)(y_true, y_pred)
        r = recall_threshold(threshold_value)(y_true, y_pred)
        bb = beta_value ** 2
        fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
        return fbeta_score
    return fbeta_score

def calculate_cm(y_true, y_pred):
    fp = np.sum((y_pred == 1) & (y_true == 0))
    tp = np.sum((y_pred == 1) & (y_true == 1))
    fn = np.sum((y_pred == 0) & (y_true == 1))
    tn = np.sum((y_pred == 0) & (y_true == 0))
    return tp, fp, fn, tn

def calculate_recall(tp, fp, fn, tn):
    return (tp)/(tp + fn)

def calculate_fallout(tp, fp, fn, tn):
    return (fp)/(fp + tn)

def calculate_fpr_tpr(y_true, y_pred):
    tp, fp, fn, tn = calculate_cm(y_true, y_pred)
    tpr = calculate_recall(tp, fp, fn, tn)
    fpr = calculate_fallout(tp, fp, fn, tn)
    return fpr, tpr

In [3]:
# earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=1, mode='auto')
# log = CSVLogger('saved_models/log_pretrained_CNN.csv')
# checkpointer = ModelCheckpoint(filepath='saved_models/pretrainedDenseNet.best.from_scratch.hdf5', verbose=1, save_best_only=True)

In [4]:
df = pd.read_csv('./dataset_information/Data_Entry_2017.csv')

diseases = [
    'Cardiomegaly','Emphysema','Effusion','Hernia','Nodule',
    'Pneumothorax','Atelectasis','Pleural_Thickening','Mass','Edema',
    'Consolidation','Infiltration','Fibrosis','Pneumonia'
]

In [5]:
# Applying one hot encoding manually
# Setting value of each disease-column 1 or 0
for disease in diseases :
    df[disease] = df['Finding Labels'].apply(lambda x: 1 if disease in x else 0)
    
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,...,Nodule,Pneumothorax,Atelectasis,Pleural_Thickening,Mass,Edema,Consolidation,Infiltration,Fibrosis,Pneumonia
0,00000001_000.png,Cardiomegaly,0,1,058Y,M,PA,2682,2749,0.143,...,0,0,0,0,0,0,0,0,0,0
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,058Y,M,PA,2894,2729,0.143,...,0,0,0,0,0,0,0,0,0,0
2,00000001_002.png,Cardiomegaly|Effusion,2,1,058Y,M,PA,2500,2048,0.168,...,0,0,0,0,0,0,0,0,0,0
3,00000002_000.png,No Finding,0,2,081Y,M,PA,2500,2048,0.171,...,0,0,0,0,0,0,0,0,0,0
4,00000003_000.png,Hernia,0,3,081Y,F,PA,2582,2991,0.143,...,0,0,0,0,0,0,0,0,0,0


In [6]:
all_image_paths = {
#     os.path.basename(x): x for x in glob(os.path.join('..', 'input','data','images*','images','*.png')),
    os.path.basename(x): x for x in glob(os.path.join('.', 'images', '*.png'))
}

print('Images found:', len(all_image_paths))

Images found: 112120


In [7]:
df['Path'] = df['Image Index'].map(all_image_paths.get)

In [8]:
labels = df[diseases].to_numpy()
files_list = df['Path'].tolist()

In [9]:
df['Path'] = df['Image Index'].map(all_image_paths.get)
files_list = df['Path'].tolist()

labelB = (df[diseases].sum(axis=1)>0).tolist()
labelB = np.array(labelB, dtype=int)

In [10]:
def path_to_tensor(img_path, shape):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=shape)
    # converting PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img) / 255
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths, shape):
    list_of_tensors = [path_to_tensor(img_path, shape) for img_path in tqdm(img_paths, desc = "Progress", ncols = 80)]
    return np.vstack(list_of_tensors)

In [11]:
# Getting the labels against each sample in all 3 dataset subsets
train_labels = labelB[ : 89600][:, np.newaxis]
valid_labels = labelB[89600 : 100800][:, np.newaxis]
test_labels = labelB[100800 : ][ : , np.newaxis]

In [12]:
# Setting some hyper-parameters
img_shape = (64, 64)
epochs = 1
batch_size = 32

In [13]:
# Loading the image-arrays into memory
train_tensors = paths_to_tensor(files_list[ : 89600], shape = img_shape)
valid_tensors = paths_to_tensor(files_list[89600 : 100800], shape = img_shape)
test_tensors = paths_to_tensor(files_list[100800 : ], shape = img_shape)

Progress: 100%|███████████████████████████| 11320/11320 [03:14<00:00, 58.22it/s]


In [14]:
base_model = DenseNet201(
    weights = 'imagenet', 
    include_top = False, 
    input_shape = train_tensors.shape[1:]
)

classifier = Sequential()
classifier.add(Flatten(input_shape = base_model.output_shape[1:]))
classifier.add(Dropout(0.2))
classifier.add(Dense(256, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(50, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(1, activation = 'sigmoid'))

model = Model(inputs=base_model.input, outputs = classifier(base_model.output))

# model.summary()

In [15]:
adam_optimizer = tf.keras.optimizers.Adam(
    learning_rate = 1e-4,
    beta_1 = 0.9,
    beta_2 = 0.999,
)

In [16]:
model.compile(
    optimizer = adam_optimizer, 
    loss = 'binary_crossentropy', 
    metrics=[
        'accuracy',
        precision_threshold(threshold = 0.5), 
        recall_threshold(threshold = 0.5), 
        fbeta_score_threshold(beta=0.5, threshold = 0.5)
    ]
)

In [17]:
train_datagen = ImageDataGenerator(
    featurewise_center=False, 
    samplewise_center=False,  
    featurewise_std_normalization=False,  
    samplewise_std_normalization=False,  
    zca_whitening=False,  
    rotation_range=10,  
    width_shift_range=0.1,  
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False 
)

In [18]:
%%timeit -n1 -r1

history = model.fit_generator(
    train_datagen.flow(train_tensors,train_labels, batch_size = batch_size),
    steps_per_epoch = len(train_tensors) // batch_size,
    validation_data = (valid_tensors, valid_labels),
    validation_steps = len(valid_tensors) // batch_size,
    epochs = epochs,
#     callbacks=[checkpointer], 
    verbose=1
#     callbacks=[checkpointer, log, earlystop], verbose=1
)

C:\Users\m7irt\anaconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


2800/2800 [==============================] - 692s 233ms/step - loss: 0.6350 - accuracy: 0.6556 - precision: 0.6398 - recall: 0.5815 - fbeta_score: 0.6206 - val_loss: 0.6181 - val_accuracy: 0.6865 - val_precision: 0.5457 - val_recall: 0.6211 - val_fbeta_score: 0.5506
11min 32s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [19]:
# model.save('dense-adam.h5')
# model.save_weights('dense-adam-weights.h5')

In [20]:
prediction = model.predict(test_tensors)

In [21]:
testing_predictions = []
for val in prediction:
    if val[0] < 0.5:
        testing_predictions.append([0])
    else:
        testing_predictions.append([1])

testing_predictions = np.array(testing_predictions)

TP, FP, FN, TN = calculate_cm(test_labels, testing_predictions)
FPR, TPR = calculate_fpr_tpr(test_labels, testing_predictions)

threshold = 0.5
beta = 0.5

accuracy = K.eval(binary_accuracy(K.variable(value=test_labels), K.variable(value=prediction)))
precision = K.eval(precision_threshold(threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))
recall = K.eval(recall_threshold(threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))
f1_score = K.eval(fbeta_score_threshold(beta = beta, threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))

print (f"Accuracy: {accuracy} \nRecall: {recall} \nSpecificity: {TN / (TN + FP)}\nPrecision: {precision} \nF1-Score: {f1_score}\n")

Accuracy: 0.675883412361145 
Recall: 0.724711000919342 
Specificity: 0.629149377593361
Precision: 0.6516160368919373 
F1-Score: 0.6650310158729553

